In [5]:
import numpy as np
from scipy.spatial import Delaunay
from preprocessing.graph_from_snapshot import build_graph_from_snapshot

In [4]:
# Load static mesh #
mesh_path = "/Users/karan94/Documents/MeshGraphNets_Airfoil/data/Airfoil"
grid = np.load(f"{mesh_path}/test_grid.npy")
cells = np.load(f"{mesh_path}/test_cells.npy")

# Load dynamic data
data = np.load(f"{mesh_path}/test_data.npy")

print(f"Grid shape: {grid.shape}, Cells shape: {cells.shape}, Data shape: {data.shape}")

Grid shape: (100, 5233, 31, 2), Cells shape: (100, 10216, 31, 3), Data shape: (100, 5233, 31, 4)


In [6]:
frame = 0
airfoil_id = 0
features = data[airfoil_id, :, frame, :3]   #
target = data[airfoil_id, :, frame + 1, :3] # next frame
node_pos = grid[airfoil_id, :, frame, :]
triangles = cells[airfoil_id, :, frame, :]
node_type = np.zeros((features.shape[0],)) # dummy

In [7]:
graph = build_graph_from_snapshot(node_pos, triangles, features, target, node_type)

In [8]:
graph

{'nodes': array([[ 1.0473521e+02,  1.3460818e+01,  1.2269930e+00,  9.9975002e-01,
         -3.6328966e-05,  0.0000000e+00],
        [ 1.1291812e+02,  1.6061655e+01,  1.2110808e+00,  9.9900001e-01,
         -1.4525375e-04,  0.0000000e+00],
        [ 1.1768428e+02,  1.6921492e+01,  1.2060925e+00,  9.9776000e-01,
         -3.2511118e-04,  0.0000000e+00],
        ...,
        [ 1.3663853e+02,  2.9403954e+00,  1.2249781e+00,  1.7549320e+01,
          3.0887530e+00,  0.0000000e+00],
        [ 1.3663853e+02,  2.9403954e+00,  1.2249781e+00,  1.1416820e+01,
          1.4055120e+01,  0.0000000e+00],
        [ 1.3663853e+02,  2.9403954e+00,  1.2249781e+00,  1.7193159e+01,
          7.9130592e+00,  0.0000000e+00]], shape=(5233, 6), dtype=float32),
 'edges': array([[-0.00078404, -0.00098731],
        [ 0.58022213, -0.05370498],
        [-0.03317043,  0.00348765],
        ...,
        [ 0.01863009, -0.02238384],
        [-0.00892198,  0.00269331],
        [-0.01995015, -0.01152593]], shape=(30898, 2